<a href="https://colab.research.google.com/github/lugsantistebanji/WCS-IA/blob/main/WCS_IA_Quetes_NLP_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge - Tweetons !


Entraîne-toi à classifier des tweets pour dire s'ils sont positifs ou négatifs. Ca pourrait être un outil très utile pour optimiser le travail d'un community manager.

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

---

1. __Importe [cet ensemble](https://github.com/murpi/wilddata/raw/master/quests/tweets.zip) de données de tweets dans un DataFrame.__

In [27]:
link = "https://github.com/murpi/wilddata/raw/master/quests/tweets.zip"
df = pd.read_csv(link, compression='zip')

In [28]:
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


---

2. __Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral).__

In [29]:
df.sentiment.unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [30]:
df_filter = df.loc[df.sentiment.isin(['negative', 'positive'])]

In [31]:
df_filter.shape

(16363, 4)

---

3. __Quel est le pourcentage de tweets positifs/négatifs ?__

In [32]:
df_filter.groupby('sentiment').sentiment.value_counts().apply(lambda x: 100*x/len(df_filter))

,count
sentiment,
negative,47.552405
positive,52.447595


---

4. __Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32 et un train_size de 0.75.__

In [33]:
X = df_filter['text']
y = df_filter['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=32, train_size=0.75)

In [34]:
X_train.shape

(12272,)

In [35]:
X_test.shape

(4091,)

---

5. __Crée un modèle vectorizer avec scikit-learn en utilisant la méthode Countvectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements.__

__Count Vectorizer__

In [36]:
cvectorizer = CountVectorizer()
X_train_CV = cvectorizer.fit_transform(X_train)
X_test_CV = cvectorizer.transform(X_test)

__TFIDF Vectorizer__

In [37]:
tfivectorizer = TfidfVectorizer()
X_train_TFI = tfivectorizer.fit_transform(X_train)
X_test_TFI = tfivectorizer.transform(X_test)

In [38]:
X_test_CV.shape

(4091, 15806)

---

6. __Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.966 pour le test d'entraînement, et 0.877 pour l'ensemble de test.__

In [39]:
modelLoR_CV = LogisticRegression()
modelLoR_TFI = LogisticRegression()

In [40]:
modelLoR_CV.fit(X_train_CV, y_train)

y_train_predict_CV = modelLoR_CV.predict(X_train_CV)
y_test_predict_CV = modelLoR_CV.predict(X_test_CV)

In [41]:
modelLoR_TFI.fit(X_train_TFI, y_train)

y_train_predict_TFI = modelLoR_TFI.predict(X_train_TFI)
y_test_predict_TFI = modelLoR_TFI.predict(X_test_TFI)

In [42]:
accuracy_train_CV = accuracy_score(y_train, y_train_predict_CV)
accuracy_test_CV = accuracy_score(y_test, y_test_predict_CV)

In [43]:
accuracy_train_TFI = accuracy_score(y_train, y_train_predict_TFI)
accuracy_test_TFI = accuracy_score(y_test, y_test_predict_TFI)

In [44]:
print(f"CountVectorizer: Score for train set {accuracy_train_CV}")
print(f"CountVectorizer: Score for test set {accuracy_test_CV}")
print(f"TFIDF: Score for train set {accuracy_train_TFI}")
print(f"TFIDF Score for test set {accuracy_test_TFI}")

CountVectorizer: Score for train set 0.9659387222946545
CountVectorizer: Score for test set 0.8772916157418724
TFIDF: Score for train set 0.9313070404172099
TFIDF Score for test set 0.8731361525299438


---

7. __Lignes avec different prediction__

In [45]:
results = pd.DataFrame(X_test.copy())
results['real_sentiment'] = y_test.values
results['predicted_sentiment_CV'] = y_test_predict_CV
results['predicted_sentiment_TFI'] = y_test_predict_TFI

In [48]:
results.loc[results['predicted_sentiment_CV'] != results['predicted_sentiment_TFI']]

,text,real_sentiment,predicted_sentiment_CV,predicted_sentiment_TFI
26241,Watching 1971 edition if Old Grey Whistle Test...,negative,positive,negative
25570,Loll whats boyfriend #2 supposed to mean then?...,negative,positive,negative
8351,I`m jumping off the nearest bridge if I can`t ...,negative,positive,negative
19103,Honestly at home alone,positive,negative,positive
17214,"bed...sorta. today was good, sara has strep th...",positive,negative,positive
...,...,...,...,...
24290,are you are a RIOT!!,negative,negative,positive
1242,"_zaboo its ok, it just hurts abit i thought i ...",negative,negative,positive
7082,"the $60,000 i spent going to college was well ...",positive,negative,positive
11322,Aw poor you..ironing! Eek! lol It`s raining h...,negative,negative,positive


---